In [ ]:
import os
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, List
from langchain.schema import Document
from dotenv import load_dotenv
from langchain_community.retrievers import ArxivRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama


# Load API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Initialize LLM and embeddings
llm = Ollama(model="llama3", temperature=0)
embeddings = HuggingFaceEmbeddings(model"all-MiniLM-L6-v2"_name=)

# Corrected retriever initialization
retriever = ArxivRetriever(
    load_max_docs=2,
    get_full_documents=True,
)

# Define the state structure
class AgentState(TypedDict):
    search: str
    documents: List[Document]
    keywords: List[str]
    answer: str

# Example documents
sample_texts = [
    "A container is a lightweight, standalone, and executable unit that includes:",
    "Containers share the host OS kernel, which makes them faster and more resource efficient compared to Virtual Machine.",
    "A Docker Image is a read-only template used to create containers.",
    "A hypervisor ecosystem refers to the complete set of tools, technologies, and services that work with a hypervisor to manage, secure, and optimize virtualized environments."
]

documents = [Document(page_content=text) for text in sample_texts]

# Build FAISS vectorstore
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(k=3)

# Node 1
def need_line(state: AgentState):
    search = state.get("search", "")
    keywords = state.get("keywords", [])
    if not keywords:
        return {**state, "answer": "", "search": search}
    answer_found = any(keyword.lower() in search.lower() for keyword in keywords)
    return {**state, "answer": str(answer_found)}

# Node 2
def check_search(state: AgentState):
    search = state.get("search", "")
    documents = retriever.invoke(search)
    return {**state, "documents": documents}

# Node 3
def answer_doc(state: AgentState):
    search = state.get("search", "")
    documents = state.get("documents", [])
    if documents:
        context = "\n\n".join([doc.page_content for doc in documents])
        prompt = f"Based on the context below, answer the question.\nContext:\n{context}\nQuestion: {search}\nAnswer:"
    else:
        prompt = f"Answer the question based on your knowledge: {search}"

    response = llm.invoke(prompt)
    return {**state, "answer": response}

# Conditional branching
def found_search(state: AgentState):
    if state.get("answer") == "True":
        return "answer_doc"
    else:
        return "search"

# Workflow
graph = StateGraph(AgentState)

graph.add_node("need_line", need_line)
graph.add_node("check_search", check_search)
graph.add_node("answer_doc", answer_doc)

graph.add_edge(START, "need_line")

graph.add_conditional_edges(
    "need_line",
    found_search,
    {
        "answer_doc": "answer_doc",
        "search": "check_search",
    },
)

graph.add_edge("check_search", "answer_doc")
graph.add_edge("answer_doc", END)

chain = graph.compile()

# Function to ask questions
def ask_question(question: str):
    initial_state = {
        "search": question,
        "documents": [],
        "answer": "",
        "keywords": ["docker", "container", "image"],
    }
    return chain.invoke(initial_state)

# Run test
question1 = "give a ruskin bond bithday"
result1 = ask_question(question1)
print(result1)




{'search': 'give a ruskin bond bithday', 'documents': [Document(id='f355f9af-d2e8-4741-9c53-c95284b2a433', metadata={}, page_content='A container is a lightweight, standalone, and executable unit that includes:'), Document(id='ee23aeed-4d0a-43c0-9909-3a42684c2690', metadata={}, page_content='A hypervisor ecosystem refers to the complete set of tools, technologies, and services that work with a hypervisor to manage, secure, and optimize virtualized environments.'), Document(id='87300848-5127-4473-93c4-21ad510b0846', metadata={}, page_content='A Docker Image is a read-only template used to create containers.'), Document(id='339b29d3-3c62-45af-b9a7-8e97f7732f2b', metadata={}, page_content='Containers share the host OS kernel, which makes them faster and more resource efficient compared to Virtual Machine.')], 'keywords': ['docker', 'container', 'image'], 'answer': "I think there may be some confusion here!\n\nThe context you provided talks about containers, hypervisors, and Docker Images.